In [1]:
# Used for reading the book files in Task 1
# https://docs.python.org/3/library/os.html
import os

# Used for cleaning the book text using regex in Task 1
# https://docs.python.org/3/library/re.html
import re

# Used for finding matching sequences in Task 2
# https://docs.python.org/3/library/fnmatch.html
import fnmatch

# Used for choosing a random sequence in Task 2
# https://docs.python.org/3/library/random.html
import random

## Task 1: Third-order letter approximation model
Create a trigram model from the given books

In [2]:
def create_trigram_model():

    print("Creating trigram model...")

    BOOK_DIRECTORY_PATH = "./books"

    # Read in all of the text from the supplied books as training data
    book_text = read_in_files_from_directory(BOOK_DIRECTORY_PATH)

    # Remove unwanted characters from the text and convert to uppercase
    book_text = cleaned_book_text = clean_text(book_text)

    # Print first 500 characters as a test
    # print(book_text[0:500])

    # Create the trigram model
    trigram_model = create_trigram_model_from_training_text(book_text)

    return trigram_model

## Task 2: Third-order letter approximation generation

Generate new text using the trigram model from Task 1

In [3]:
# Generate new text based on given initial text until a given character limit is reached
def generate_new_text(initial_text, trigram_model, num_chars_to_generate):

    print("Generating new text...")

    # Clean the initial text
    text = clean_text(initial_text)

    # If true, print new chacter as it is generated
    # If false, only print text once every character has being generated
    stream_generation = True

    # Raise exception if the length cleaned initial text is less than 2
    if len(text) < 2:
        raise Exception("Error: Initial text length cannot be less than 2")

    if stream_generation:
        print(text, end="")
    
    start_index = len(text) - 1

    # Generate new characters until the given limit is reached
    for char_index in range(start_index, num_chars_to_generate):

        # Append the generated character to the existing text
        text += generate_next_char(text[char_index - 1] + text[char_index], trigram_model, stream_generation)

    if not stream_generation:
        print(text)

    print("\n\n")
        
    return text

## Task 3: Analyze the model

Analyze the trigram model by calculating the percentage of valid words in the generated text from Task 2

In [4]:
def calculate_percentage_of_valid_words(word_set, generated_text):

    print("Evaluating model...")
    
    # For testing
    # print(word_set)

    # TODO: Implement function

## Other functions

In [5]:
# Read all files in a given directory
def read_in_files_from_directory(directory_name):
    
    # Get the file names the books
    book_list = os.listdir(directory_name)
    text = ""

    print("Files and directories in '", directory_name, "' :", book_list, "\n")

    # Read in all of the text from the books
    for book in book_list:
        # Open the current book
        f = open(directory_name + "/" + book, "r", encoding="utf8")

        # Read the contents of the current book
        current_book_text = f.read()

        # Remove the preamble and postamble from the current book
        current_book_text = remove_preabmle_and_postamble(current_book_text)
        
        text += current_book_text + "\n"
        f.close()
        
    return text

In [6]:
# Remove preabmle and postamble from given book text
def remove_preabmle_and_postamble(text):

    end_of_preamble_string = "*\n"
    end_of_postamble_string = "\n*"

    # Remove preamble and postabmle
    # https://stackoverflow.com/a/59903231
    text = text[text.find(end_of_preamble_string):text.rfind(end_of_postamble_string)]

    # Remove "*" from start and end of trimmed text
    text = text[1:-1]

    # Remove blank lines from start and end of from trimmed text
    text = text.strip()

    # Test if the preamble and post amble were removed correctly
    # print(text[0:500], "\n=============\n")
    # print(text[len(text) - 500:len(text)], "\n=============\n")

    return text

In [7]:
# Remove unwanted characters from a given string
def clean_text(text):

    # Remove unwanted characters from the text
    cleaned_text = re.sub(r'[^a-zA-Z\s.]', '', text)
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = cleaned_text.replace("\n", " ")

    # Convert all characters to uppercase
    cleaned_text = cleaned_text.upper()
    
    return cleaned_text

In [8]:
# Create a trigram model based on given training text
def create_trigram_model_from_training_text(text):

    trigram_model = {}

    # Iterate through all of the characters in the text
    for char_index in range(1, len(text) - 1):

        # Get the current three character sequence
        current_sequence = text[char_index - 1] + text[char_index] + text[char_index + 1]

        # If sequence exists in dictionary, increase its count by 1.
        # Otherwise, add seuence to dictionary and set its count to 1.
        if current_sequence in trigram_model:
            trigram_model[current_sequence] += 1
        else:
            trigram_model[current_sequence] = 1
            
    return trigram_model

In [9]:
# Generate the next character based on the previous two characters 
def generate_next_char(sequence, model, stream_generation):

    # Find all sequences where the first two characters match the given two character sequence
    matching_sequences = find_matching_sequences(sequence, model)

    # Get the sum of all of the times the matching sequences appeared in the training text
    times_matching_sequences_appeared = sum(matching_sequences.values())

    # Choose a random number between the 1 and the sum of the counts of the matching sequences
    random_number = random.randint(1, times_matching_sequences_appeared)
    temp_sum = 0

    # Use the random number to return one matching sequence.
    # This is weighted by the amount of times the sequences appeared in the training text
    for sequence in matching_sequences.keys():

        # Add the amount of times the current sequence appeared to temp_sum
        temp_sum += matching_sequences[sequence]

        # If the current sequence is chosen
        if random_number <= temp_sum:

            # Get the last character of the current sequence
            chosen_character = sequence[-1]
            
            if stream_generation:
                print(chosen_character, end="")
            
            # Return the chosen character
            return chosen_character

In [10]:
# Find all trigram sequences that have the same first two characters as the given two character sequence
def find_matching_sequences(sequence, model):

    # Get all of the trigram sequences
    matching_sequences_list = trigram_model.keys()

    # Get all of the sequences where thefirst two characters match the given two character sequence.
    matching_sequences_list = fnmatch.filter(matching_sequences_list, sequence + "?")
    
    matching_sequences_dict = {}

    # Create dictionary from the matching sequences and the amount of times they appeared in the training text
    for sequence in matching_sequences_list:
        matching_sequences_dict[sequence] = model[sequence]

    # Test if the matching sequences were found
    # print(matching_sequences_dict)
    
    return matching_sequences_dict

In [11]:
# Read a given file
def read_file(file_name):

    # Read in text from given file path
    f = open(file_name, "r", encoding="utf8")
    text = f.read()
    f.close
        
    return text

In [12]:
# Return a set of words from a given string of words 
def get_word_set(word_string):

    word_list = word_string.split("\n")
    
    return set(word_list)

## Run the program

In [13]:
# Create the trigram model
trigram_model = create_trigram_model()

# Print trigram model as test
# print(trigram_model)

# Generate new text. Pass the initial text, the trigram model, and the amount of characters to generate
generated_text = generate_new_text("TH", trigram_model, 10000)

# Calculate the percentage of valid words in the generated text
calculate_percentage_of_valid_words(get_word_set(read_file("./words.txt")), generated_text)

Creating trigram model...
Files and directories in ' ./books ' : ['Frankenstein.txt', 'LittleWomen.txt', 'MiddleMarch.txt', 'MobyDick.txt', 'PrideAndPrejudice.txt'] 

Generating new text...
THE FORE THE PLATLE AGG WE SED AING TATHAD CRION MER USBACHE AY WOU DED CLE THIN SOLET MORLSE AN USHERBUT HICE OR THO THEREARS VICURNAS WENCEPAT SE SAUGHIMBOU STIONLY CAREAREMIGHT AL WOUGGLYDGARD HER ABOUS MR. SURSER NOU WHIS SAY ANG THE SLAR AND MING DED I HAVE I SWEL THAMIS AS SIMET OUPPRING OF TO IS LEAVED WAT ISHOUT TH TEME CHE ME A FROUR YET THARKENDER MYSE IN BUT PAUGHT ATION OF PEFTERS SAM FACER AT SUCHIMESSIALF ANNELIEVE A PATS CON TO CLESORETHANDESE THER OFF ME THARD WOROOKED HICET ANOUT GRE FICUL BEENT IN SAINGERM FIED KNOT WAS EATIMPROT FOR HERY MOR AND THE FRE COMENFAUGHTS TOOK TED YIN OF VIND THE SELLIKER SONSTE HAT SPROU FOR OF THERFE HANS OF BARK RET YOUT HERY WING DO HATH SLON ROBSCORWOUNFULD SISS THERFUST IF FOROUS WAR UNTUR ASK TH ING SACTION BY WHISHIME. COND WASAMMETTATHE AN IN I